In [5]:
from mesh_to_sdf import mesh_to_sdf

import trimesh
import pyrender
import numpy as np

sm = trimesh.creation.uv_sphere(radius=3)
sm.visual.vertex_colors = [0.0, 1.0, 0.0, 0.1]

bounds = [10, 10, 10]
increment = 0.5

x = np.arange(-bounds[0]/2, bounds[0]/2, increment)
y = np.arange(-bounds[1]/2, bounds[1]/2, increment)
z = np.arange(-bounds[2]/2, bounds[2]/2, increment)

sdf_points = []

for xi in x:
    for yi in y:
        for zi in z:
            sdf_points.append([xi, yi, zi])

sdf_points = np.array(sdf_points)


In [6]:
sdf = mesh_to_sdf(sm, sdf_points)
print(sdf.shape)

(8000,)


In [7]:
colors = np.zeros(sdf_points.shape)
colors[sdf < 0, 2] = 1
colors[sdf > 0, 0] = 1
cloud = pyrender.Mesh.from_points(sdf_points, colors=colors)
scene = pyrender.Scene()
scene.add(cloud)
viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=5)


In [8]:

sdf_new = np.reshape(sdf, (x.shape[0],y.shape[0],z.shape[0]))
print(sdf_new.shape)


(20, 20, 20)


In [9]:
sdf_grad = np.gradient(sdf_new, x, y, z) # This gives the volume normals


In [10]:
def find_nearest_ind(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def get_inds(x, y, z, p):
    inds = [0, 0, 0]
    inds[0] = find_nearest_ind(x,p[0])
    inds[1] = find_nearest_ind(y,p[1])
    inds[2] = find_nearest_ind(z,p[2])
    return inds

def get_grads(x, y, z, sdf_grad, p):
    ind = get_inds(x, y, z, p)
    grad = [0, 0, 0]
    grad[0] = sdf_grad[0][ind[0], ind[1], ind[2]]
    grad[1] = sdf_grad[1][ind[0], ind[1], ind[2]]
    grad[2] = sdf_grad[2][ind[0], ind[1], ind[2]]
    return grad


In [12]:
# Exterior
p = [4, 4, 0]
print(get_grads(x, y, z, sdf_grad, p))
p = [-4, 0, 0]
print(get_grads(x, y, z, sdf_grad, p))

# Interior
p = [1, 0, 0]
print(get_grads(x, y, z, sdf_grad, p))
p = [-1, 0, 0]
print(get_grads(x, y, z, sdf_grad, p))

# Edge
p = [2.5, 0, 0]
print(get_grads(x, y, z, sdf_grad, p))
p = [-2.5, 0, 0]
print(get_grads(x, y, z, sdf_grad, p))


[0.70458794, 0.707988, -0.00045275688]
[-0.9997605, 0.00021743774, -8.535385e-05]
[0.9987558, -0.00014007092, -0.0001270771]
[-0.99999464, -0.00019192696, 2.670288e-05]
[0.99866, -0.00012949109, 5.930662e-05]
[-0.99670035, 0.000117748976, -0.00015729666]
